# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [868]:
import numpy as np
import pandas as pd
import os
from datetime import date

DATA_FOLDER = 'Data'

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### A) Pre-processing data

In [1004]:
GUINEA_PATH  = DATA_FOLDER + '/ebola/guinea_data/'
LIBERIA_PATH = DATA_FOLDER + '/ebola/liberia_data/'
SL_PATH      = DATA_FOLDER + '/ebola/sl_data/'

"""Read data"""

GUINEA_DATAS  = [pd.read_csv(GUINEA_PATH  + file) for file in os.listdir( GUINEA_PATH ) if file.endswith(".csv")]
LIBERIA_DATAS = [pd.read_csv(LIBERIA_PATH + file) for file in os.listdir( LIBERIA_PATH )if file.endswith(".csv")]
SL_DATAS      = [pd.read_csv(SL_PATH      + file) for file in os.listdir( SL_PATH )     if file.endswith(".csv")]

In [1005]:
"""Concatenate all files of each country and replace missing value by zero"""

GUINEA_dframes  = pd.concat(GUINEA_DATAS)
LIBERIA_dframes = pd.concat(LIBERIA_DATAS)
SL_dframes      = pd.concat(SL_DATAS)

GUINEA_dframes  = GUINEA_dframes .fillna(0)
LIBERIA_dframes = LIBERIA_dframes.fillna(0)
SL_dframes      = SL_dframes     .fillna(0)

In [1006]:
"""First of all, take a look at data header and decide which columns are useful"""

GUINEA_dframes  = GUINEA_dframes [['Date', 'Description', 'Totals']]
LIBERIA_dframes = LIBERIA_dframes[['Date', 'Variable', 'National']]
SL_dframes      = SL_dframes     [['date', 'variable', 'National']]

In [1007]:
"""Rename labels for having the same format"""

LIBERIA_dframes = LIBERIA_dframes.rename(index=str, columns={"Variable": "Description", "National": "Totals"})
SL_dframes      = SL_dframes.     rename(index=str, columns={"date": "Date","variable": "Description", "National": "Totals"})

In [1008]:
"""Change format Date of data provided by LIBERIA"""
"""Change 6/16/2014 (MM/DD/YYYY) to 2014-06-16 (YYYY-MM-DD)"""
"""Attention with data '6/16/17' 17 means 2017 :)"""

def changeDateFormat(date):
    string = date.split('/')
    if (len(string[0]) == 1) : string[0] = '0' + string[0]
    if (len(string[1]) == 1) : string[1] = '0' + string[1]
    if (len(string[2]) == 2) : string[2] = '20'+ string[2]
    return string[2] + '-' + string[0] + '-' + string[1]

LIBERIA_dframes['Date'] = LIBERIA_dframes['Date'].apply(lambda x : changeDateFormat(str(x)))

In [1009]:
"""Parsing string Date to object TimeDate"""

def parseDate(string):
    result = pd.to_datetime(string, format='%Y-%m-%d', errors='coerce')
    if (type(result) is pd.tslib.NaTType):
        print('Problem man !!!')
    return result

GUINEA_dframes ['Date'] = GUINEA_dframes ['Date'].apply(lambda x : parseDate(str(x)))
LIBERIA_dframes['Date'] = LIBERIA_dframes['Date'].apply(lambda x : parseDate(str(x)))
SL_dframes     ['Date'] = SL_dframes     ['Date'].apply(lambda x : parseDate(str(x)))

In [1010]:
"""Insert an extra column Country"""

GUINEA_dframes .insert(3, 'Country', 'GUINEA')
LIBERIA_dframes.insert(3, 'Country', 'LIBERIA')
SL_dframes     .insert(3, 'Country', 'SL')

In [1011]:
"""Set indexes"""

GUINEA_dframes.index  = range(len(GUINEA_dframes))
LIBERIA_dframes.index = range(len(LIBERIA_dframes))
SL_dframes.index      = range(len(SL_dframes))


In [1012]:
"""Drop all non-integer elements in column 'Totals'"""


GUINEA_dframes  = GUINEA_dframes[ GUINEA_dframes  ['Totals'].apply(lambda x: str(x).isdigit())]
#LIBERIA_dframes = LIBERIA_dframes[ LIBERIA_dframes['Totals'].apply(lambda x: x.isdigit() or str(x).isdigit())]
SL_dframes      = SL_dframes[ SL_dframes          ['Totals'].apply(lambda x: str(x).isdigit())]


In [1013]:
"""Convert all integer of type string into integer"""

GUINEA_dframes ['Totals'] = pd.to_numeric( GUINEA_dframes ['Totals'], errors='coerce')
LIBERIA_dframes['Totals'] = pd.to_numeric( LIBERIA_dframes['Totals'], errors='coerce')
SL_dframes     ['Totals'] = pd.to_numeric( SL_dframes     ['Totals'], errors='coerce')


### B) Computation

#### GUINEA
In reports of GUINEA, there are 2 types of new cases: 
* 'Total new cases' (with a litle 'n' in new cases).
* 'New cases' (with a capital 'N' in New cases) which is relevant because we can take into account the case 'New cases of confirmed..' too.



In [1014]:
print('The daily average per month of new cases')

"""Find key New cases"""
GUINEA_newcases = GUINEA_dframes[ [('New cases' in desc) for desc in GUINEA_dframes.Description]]

"""Groupby Month for being able to aggregate in Totals"""
daily = GUINEA_newcases.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a month"""
days = GUINEA_newcases.groupby(GUINEA_newcases.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]


"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'new cases' on month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))


The daily average per month of new cases
The average 'new cases' on month 8 is 25.8
The average 'new cases' on month 9 is 20.0625
The average 'new cases' on month 10 is 36.0


I got a litle confuse here... how do we define a death case

In [1015]:
print('The daily average per month of deaths')

GUINEA_deaths = GUINEA_dframes[ [('confirmed' in desc and 
                                  'deaths'    in desc and 
                                  'Total deaths (confirmed + probables + suspects)' not in desc or    
                                  (   'deaths registered' in desc and
                                      '(confirmed)'   not in desc and
                                      '(probables)'   not in desc and
                                      '(suspects)'    not in desc 
                                  )
                                ) for desc in GUINEA_dframes.Description]]

"""Groupby Month for being able to aggregate in Totals"""
daily = GUINEA_deaths.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a month"""
days = GUINEA_deaths.groupby(GUINEA_deaths.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]


"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'deaths' of month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))

The daily average per month of deaths
The average 'deaths' of month 8 is 349.6
The average 'deaths' of month 9 is 485.8125
The average 'deaths' of month 10 is 612.0


#### LIBERIA

In [1016]:
""""""
print('The daily average per month of deaths')


LIBERIA_newcases = LIBERIA_dframes[ [('New cases' in desc  or 
                                      'New case'  in desc  or
                                      'New Case'  in desc  or
                                      'New Cases' in desc 
                                     ) for desc in LIBERIA_dframes.Description]]


"""Groupby Month for being able to aggregate in Totals"""
daily = LIBERIA_newcases.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a specific month"""
days = LIBERIA_newcases.groupby(LIBERIA_newcases.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]


"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'new cases' on month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))

The daily average per month of deaths
The average 'new cases' on month 6 is 5.714285714285714
The average 'new cases' on month 7 is 8.545454545454545
The average 'new cases' on month 8 is 37.22222222222222
The average 'new cases' on month 9 is 63.833333333333336
The average 'new cases' on month 10 is 45.56
The average 'new cases' on month 11 is 26.466666666666665
The average 'new cases' on month 12 is 5178.555555555556


In [1017]:
LIBERIA_deaths = LIBERIA_dframes[ [( 'deaths' in desc  or 
                                     'death'  in desc  and                                     
                                       'Total death/s in confirmed cases' not in desc and
                                    'Total death/s in probable cases' not in desc and
                                    'Total death/s in suspected cases' not in desc 
                                   ) for desc in LIBERIA_dframes.Description]]

"""Groupby Month for being able to aggregate in Totals"""
daily = LIBERIA_deaths.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a specific month"""
days = LIBERIA_deaths.groupby(LIBERIA_deaths.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]


"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'deaths' on month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))

The average 'deaths' on month 6 is 36.714285714285715
The average 'deaths' on month 7 is 109.81818181818181
The average 'deaths' on month 8 is 592.0
The average 'deaths' on month 9 is 1533.2916666666667
The average 'deaths' on month 10 is 2557.24
The average 'deaths' on month 11 is 2948.133333333333
The average 'deaths' on month 12 is 3229.222222222222


#### SL

In [1018]:
print('The daily average per month of deaths')


SL_newcases = SL_dframes[ [('new_' in desc and 'deaths' not in desc and
                            'new_negative' not in desc
                                     ) for desc in SL_dframes.Description]]
"""Groupby Month for being able to aggregate in Totals"""
daily = SL_newcases.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a specific month"""
days = SL_newcases.groupby(SL_newcases.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]


"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'new cases' on month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))

The daily average per month of deaths
The average 'new cases' on month 8 is 289.05
The average 'new cases' on month 9 is 408.44827586206895
The average 'new cases' on month 10 is 898.0
The average 'new cases' on month 11 is 334.3
The average 'new cases' on month 12 is 936.6


In [1019]:
SL_deaths = SL_dframes[ [('deaths' in desc or 'Deaths' in desc
                                     ) for desc in SL_dframes.Description]]


"""Groupby Month for being able to aggregate in Totals"""
daily = SL_deaths.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a specific month"""
days = SL_deaths.groupby(SL_deaths.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]

"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'deaths' on month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))

The average 'deaths' on month 8 is 0.0
The average 'deaths' on month 9 is 24.413793103448278
The average 'deaths' on month 10 is 407.42857142857144
The average 'deaths' on month 11 is 145.6
The average 'deaths' on month 12 is 349.4


#### Merging 3 frames

In [1020]:
df            = pd.concat([GUINEA_dframes, LIBERIA_dframes, SL_dframes])
df.index      = range(len(df))
df

,Date,Description,Totals,Country
0,2014-08-04,New cases of suspects,5.0,GUINEA
1,2014-08-04,New cases of probables,0.0,GUINEA
2,2014-08-04,New cases of confirmed,4.0,GUINEA
3,2014-08-04,Total new cases registered so far,9.0,GUINEA
4,2014-08-04,Total cases of suspects,11.0,GUINEA
5,2014-08-04,Total cases of probables,133.0,GUINEA
6,2014-08-04,Total cases of confirmed,351.0,GUINEA
7,2014-08-04,Cumulative (confirmed + probable + suspects),495.0,GUINEA
8,2014-08-04,New deaths registered today,2.0,GUINEA
9,2014-08-04,New deaths registered today (confirmed),2.0,GUINEA


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [2]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [3]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

Name,Labels,Units,Levels,Storage,NAs
pclass,,,3,integer,0
survived,Survived,,,double,0
name,Name,,,character,0
sex,,,2,integer,0
age,Age,Year,,double,263
sibsp,Number of Siblings/Spouses Aboard,,,double,0
parch,Number of Parents/Children Aboard,,,double,0
ticket,Ticket Number,,,character,0
fare,Passenger Fare,British Pound (\243),,double,1
cabin,,,187,integer,0


For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here